In [1]:
import numpy as np
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
import stars.rto.bazargan_vkc as vkc

%load_ext autoreload
%autoreload 2

In [2]:
# Functions to read in files

def parse_thermo_file(filename):
    with open(filename) as f:
        content = f.readlines()
    
    content_bools = [c.split(',')[0]=='Sample Number' for c in content]
    skiprows = content_bools.index(True)
        
    df_temp_init = pd.read_table(filename, sep=',', skiprows=skiprows+1, header=None, engine='python')
    df_temp = pd.DataFrame()
    df_temp['Temperature'] = df_temp_init[[2,3,4]].max(axis=1)
    df_temp['Time'] = pd.to_datetime(df_temp_init[1])
    df_temp['Time'] = (df_temp.Time - df_temp.Time.min()).dt.total_seconds() 
    
    return df_temp
    
def parse_gas_file(filename):

    with open(filename) as f:
        content = f.readlines()

    line0 = content[0].split(';')
    line1 = content[1].split(';')

    if len(line0) != len(line1):
        skiprows = 1
    else:
        skiprows = 0

    # Get columns for compositions
    line1stripped = [l.strip() for l in line1]
    O2ind = line1stripped.index('Oxygen')
    COind = line1stripped.index('CO')
    CO2ind = line1stripped.index('CO2')
    CH4ind = line1stripped.index('CH4')

    # Load composition data 
    df_comp_init = pd.read_table(filename, sep=';', skiprows=skiprows, header=None, engine='python')

    df_comp = pd.DataFrame()

    df_comp['Time'] = pd.to_datetime(df_comp_init[1], errors='coerce')
    df_comp['Time'] = (df_comp.Time - df_comp.Time.min()).dt.total_seconds() 
    df_comp['O2'] = pd.to_numeric(df_comp_init[O2ind+1], errors='coerce')
    df_comp['CO'] = pd.to_numeric(df_comp_init[COind+1], errors='coerce')
    df_comp['CO2'] = pd.to_numeric(df_comp_init[CO2ind+1], errors='coerce')
    df_comp['CH4'] = pd.to_numeric(df_comp_init[CH4ind+1], errors='coerce')
    
    df_comp.dropna(inplace=True)
    
    return df_comp


def parse_RTOexp_file(expdir):
    '''
    Input:
        expdir - directory with files from experiment. This directory must contain 1 file ending in 
            'temp.txt' and another in 'comp.txt' for the code to find it correctly
    
    Output:
        df_out - pandas.DataFrame object containing Time, Temperature, and gas effluence data
    '''
    
    # Parse temperature data file
    temp_file_list = [name for name in os.listdir(hrdirname) if name[-8:].lower()=='temp.txt']
    df_temp = parse_thermo_file(os.path.join(hrdirname, temp_file_list[0]))
    
    # Parse experimental data file
    comp_file_list = [name for name in os.listdir(hrdirname) if name[-8:].lower()=='comp.txt']
    df_comp = parse_gas_file(os.path.join(hrdirname, comp_file_list[0]))

    # Assemble final data frame
    df_out = pd.DataFrame()
    df_out['Time'] = df_temp.Time
    df_out['Temperature'] = df_temp.Temperature
    df_out['O2'] = pd.Series(np.interp(df_temp.Time.values, df_comp.Time.values, df_comp.O2.values))
    df_out['CO'] = pd.Series(np.interp(df_temp.Time.values, df_comp.Time.values, df_comp.CO.values))
    df_out['CO2'] = pd.Series(np.interp(df_temp.Time.values, df_comp.Time.values, df_comp.CO2.values))
    df_out['CH4'] = pd.Series(np.interp(df_temp.Time.values, df_comp.Time.values, df_comp.CH4.values))
    
    return df_out


def parse_STARS_files(expdir):
    '''
    Input:
        expdir - directory with files from experiment. This directory must contain 1 file ending in 
            'temp.txt' and another in 'comp.txt' for the code to find it correctly
    
    Output:
        df_out - pandas.DataFrame object containing Time, Temperature, and gas effluence data
    '''
        
    # Parse STARS output data
    temp_file_list = [name for name in os.listdir(hrdirname) if name[-4:].lower()=='.irf']
    KC = vkc.BazarganVKC(folder_name = hrdirname, input_file_name = os.path.join(hrdirname,temp_file_list[0][:-4]))
    KC.parse_stars_output()
    t = KC.t

    # Assemble final data frame
    df_out = pd.DataFrame()
    df_out['Time'] = pd.Series(t)
    df_out['Temperature'] = pd.Series(KC.SPEC_VALS[:,11] - 272.15)
    df_out['O2'] = pd.Series(np.interp(t, t, -KC.SPEC_VALS[:,7]))
    df_out['CO'] = pd.Series(np.interp(t, t, KC.SPEC_VALS[:,5]))
    df_out['CO2'] = pd.Series(np.interp(t, t, KC.SPEC_VALS[:,6]))
    
    return df_out


In [3]:
# dataset_names = ['chichimene', 'utsil', 'tekel']
dataset_names = ['synthetic']

# Loop over datasets to convert
for d in dataset_names:
    
    dirname = 'datasets/'+d
    experiment_names = [name for name in os.listdir(dirname) if os.path.isdir(os.path.join(dirname, name))]
    
    # Loop over experiments to convert
    for exp in experiment_names:
        expdirname = 'datasets/'+ d + '/' + exp
        hr_names = [name for name in os.listdir(expdirname) if os.path.isdir(os.path.join(expdirname, name)) 
                    and name[-5:].lower()=='c_min']
        hrs = [hr[:-5] for hr in hr_names]
        
        # Loop over heating rates
        for hr in hr_names:
            hrdirname = os.path.join(expdirname, hr)
            print(hrdirname)
            
            if d == 'synthetic':
                df_out = parse_STARS_files(hrdirname)
            else:
                df_out = parse_RTOexp_file(hrdirname)
                
            df_out.to_excel(os.path.join(expdirname, hr+'.xls'))
    


datasets/synthetic/25Cstart/31C_min


In [ ]:
plt.plot(df_out.Time.values, df_out.O2.values)
plt.show()